In [ ]:
import torch
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
torch.cuda.device_count()

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
torch.cuda.device_count()

In [ ]:
!CUDA_VISIBLE_DEVICES=1
torch.cuda.device_count()

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=1
torch.cuda.device_count()

In [ ]:
# The only way to do it is to launch jupyter this way:

import torch
torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name()

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(2)

In [ ]:
experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "CIFAR_phylogeny_tripletloss_new_archi_full"
i=0
device = 0
experiment_params = {"image_path": "cifar-100-python", "suffix": None, "img_res": 32, "augmented": True, "batchSize": 64, "learning_rate": 0.01, "numOfTrials": 2, "fc_layers": 1, "pretrained": True, "epochs": 30, "patience": 12, "optimizer": "adabelief", "scheduler": "cosine", "weightdecay": 0.0005, "modelType": "BB", "lambda": 10, "tl_model": "preResNet", "link_layer": "avgpool", "adaptive_smoothing": True, "adaptive_lambda": 0.1, "adaptive_alpha": 0.1, "tripletEnabled": True, "tripletSamples": 3, "tripletSelector": "semihard", "tripletMargin": 0.3, "phylogeny_loss": False, "displayName": "CIFARpretrained4-Triplet-Cos", "noSpeciesBackprop": False, "phylogeny_loss_epsilon": 0.03}




# experiment_params = {"image_path": "cifar-100-python", "suffix": None, "img_res": 32, "augmented": True, "batchSize": 64, "learning_rate": 0.001, "numOfTrials": 1, "fc_layers": 1, "pretrained": True, "epochs": 30, "patience": 12, "optimizer": "adabelief", "scheduler": "plateau", "weightdecay": 0.0005, "modelType": "BB", "lambda": 10, "tl_model": "preResNet", "link_layer": "avgpool", "adaptive_smoothing": True, "adaptive_lambda": 0.1, "adaptive_alpha": 0.1, "tripletEnabled": False, "tripletSamples": 3, "tripletSelector": "semihard", "tripletMargin": 0.2, "phylogeny_loss": False, "displayName": "CIFARpretrained4-BB", "noSpeciesBackprop": False, "phylogeny_loss_epsilon": 0.03}

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from myhelpers import cifar_dataLoader
from HGNN.train import CNN
from HGNN.train.configParser import getModelName
from configParser import ConfigParser, getModelName
import os
import torch
from myhelpers.resnet_cifar2 import cifar100
import numpy as np
from myhelpers.preresnet_cifar import resnet as preresnet_cifar
import random
# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(121)

config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = config_parser.fixExperimentParams(experiment_params)
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))

train_loader, validation_loader, test_loader = datasetManager.getLoaders()
architecture = {
    "fine": len(train_loader.dataset.csv_processor.getFineList()),
    "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
}

# from myhelpers.resnet_cifar import cifar_resnet56

# # modelName = getModelName(experiment_params, i)
# print(modelName)
# modelName = "models/b07640aa2d0c1aacb31f7bcda75ac772ad8bf53db3dacc06d2280a98"
# trialName = os.path.join(experimentPathAndName, modelName)
# # initModelPath = CNN.getInitModelFile(experimentPathAndName)
# finalModelPath = CNN.getModelFile(trialName)
# model.load_state_dict(torch.load(finalModelPath))

# model = cifar_resnet56(pretrained='cifar100')
model = CNN.create_model(architecture, experiment_params, device=device)
# model = cifar100(128, pretrained=True)
# model = preresnet_cifar(dataset='cifar100', inpt_size=32, pretrained=True)
# CNN.trainModel(train_loader, validation_loader, experiment_params, model, "hello", test_loader, device=device)
# print(model)

predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, device=device)
predlist, lbllist = CNN.getPredictions(predlist, lbllist)
if device is not None:
    predlist = predlist.cpu()
    lbllist = lbllist.cpu()   
print(f1_score(lbllist, predlist, average='macro'))
print(f1_score(lbllist, predlist, average='micro'))
print(accuracy_score(lbllist, predlist))

# with torch.no_grad():
#     rand_input = torch.rand(2, 3, model.network_fine.img_res, model.network_fine.img_res)
#     if model.network_fine.device is not None:
#         rand_input = rand_input.cuda()
#     out_ = model.network_fine(rand_input)

model = CNN.create_model(architecture, experiment_params, device=device)

predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, device=device)
predlist, lbllist = CNN.getPredictions(predlist, lbllist)
if device is not None:
    predlist = predlist.cpu()
    lbllist = lbllist.cpu()   
print(f1_score(lbllist, predlist, average='macro'))
print(f1_score(lbllist, predlist, average='micro'))
print(accuracy_score(lbllist, predlist))


model = model.network_fine

predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, device=device)
predlist, lbllist = CNN.getPredictions(predlist, lbllist)
if device is not None:
    predlist = predlist.cpu()
    lbllist = lbllist.cpu()   
print(f1_score(lbllist, predlist, average='macro'))
print(f1_score(lbllist, predlist, average='micro'))
print(accuracy_score(lbllist, predlist))






In [ ]:
images = next(iter(train_loader))
plt.imshow(np.transpose(images['image'][0].cpu().detach().numpy(), (1, 2, 0)))

In [ ]:
import torch
import torchvision.models as models
learning_rate = 0.00001
scheduler_gamma = 0.1
scheduler_patience = 4
epochs = 40

model = models.resnet50(pretrained=True)
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, scheduler_patience, eta_min=learning_rate*scheduler_gamma)

for i in range(epochs):
    print(scheduler.get_last_lr())
    scheduler.step() 


In [ ]:
import torch

adaptive_alpha=0.5
adaptive_lambda=0.1
fine_loss=[torch.tensor(25)]
other_losses ={
    'layer2': torch.tensor(2),
    'layer4': torch.tensor(2)
}

from myhelpers.adaptive_smoothing import get_lambdas

for fine_loss_ in fine_loss:
    print(get_lambdas(adaptive_alpha, adaptive_lambda, fine_loss_, other_losses =other_losses))



In [ ]:
from torchvision import models
model = models.resnet18()
print(model)

In [ ]:

from HGNN.train import CNN

experiment_params = {"image_path": "cifar-100-python", 
                     "suffix": None, 
                     "img_res": 32,
                     "augmented": True, 
                     "batchSize": 64,
                     "learning_rate": 0.01,
                     "numOfTrials": 2,
                     "fc_layers": 1, "pretrained": True, "epochs": 30, "patience": 12,
                     "optimizer": "adabelief", "scheduler": "cosine", "weightdecay": 0.0005,
                     "modelType": "BB", "lambda": 10, "tl_model": "ResNet18", "link_layer": "avgpool", "adaptive_smoothing": True, "adaptive_lambda": 0.1, "adaptive_alpha": 0.1, "tripletEnabled": True, "tripletSamples": 3, "tripletSelector": "semihard", "tripletMargin": 0.3, "phylogeny_loss": False, "displayName": "CIFARpretrained4-Triplet-Cos", "noSpeciesBackprop": False, "phylogeny_loss_epsilon": 0.03}

architecture = {
    "fine": 50,
    "coarse" : None
}


model = CNN.create_model(architecture, experiment_params)

print(model)

In [ ]:
import torch
import torch.nn.functional as F

embeddings = torch.tensor([[4., 3., 0.],[4., 5., 6.]])
F.normalize(embeddings, p=2, dim=-1)

In [ ]:
embeddings = torch.tensor([[4.],[4.]])
(embeddings-4).eq(torch.zeros_like(embeddings)).all()

In [ ]:
import torch
from random import choice

a = torch.tensor([[1],[2]])

print(torch.FloatTensor(list(map(lambda x : choice([i for i in range(0,3) if i not in [x]]), a))).view(-1,1))

In [ ]:
print(torch.__version__)

In [ ]:
from taxonomy import Taxonomy
from HGNN.train.CSV_processor import CSV_processor

fine_csv_scientificName_header = "scientificName"
fileNameAndPath = "/raid/elhamod/Fish/Curated4/Easy_30/cleaned_metadata.tre"
data_root = "/raid/elhamod/Fish/Curated4/Easy_30/"
suffix=""

processor = CSV_processor(data_root, suffix, verbose=True)

df_nodupes = processor.fine_csv[fine_csv_scientificName_header].drop_duplicates() # Will probably need more processing to deal with small letter...etc
node_ids = df_nodupes.tolist()

t = Taxonomy(node_ids, fileNameAndPath, verbose=True)

# t.get_total_distance()
species = node_ids[33]
print('species',species)
t.get_siblings_by_name(species, 0.5, get_ottids = False, verbose=True)


In [ ]:
# used to render the tree beautifully
!pip install pyqt5
# set flag for rendering
import os
os.environ['QT_QPA_PLATFORM']='offscreen'

import ete3
ts = ete3.TreeStyle()
ts.show_branch_length=True

t.tree.render('%%inline', tree_style=ts)
# t.tree.show()

In [ ]:
a = {}

def add(a):
    a['1'] = 2
    
add(a)
a

In [ ]:
s = [0,1,2]

s[0:3]

In [ ]:
import torch
from sklearn.preprocessing import MultiLabelBinarizer

sisters_indices = [[1, 2],[3,1,2]]
mlb = MultiLabelBinarizer(range(5))
mlb.fit_transform(sisters_indices)

In [ ]:
loss = torch.nn.BCEWithLogitsLoss()
input = torch.randn((2,3), requires_grad=True)
target = torch.tensor([[1.,0.,1.],[0.,1.,0.]])
output = loss(input, target)
output.backward()

In [ ]:
from HGNN.train import CNN

architecture
model = CNN.create_model(architecture, experiment_params, device=device)
CNN.loadModel(model, savedModelName, device=None)

In [ ]:
import torch

model = torch.load('/raid/elhamod/Fish/experiments/Fish30-5run-testPhyloNN/models/9bad7cb5922c83f40dc03a781629256c776a0ad5b44039b9099f47d8/finalModel.pt')

In [ ]:
print(model)

In [ ]:
model.network[4][1].conv2.weight

In [ ]:
phyloDistances=[0.75, 0.5, 0.25]
phyloDistances.insert(0, 1)
loss_name='1distance'
distance_indx = [idx for idx, element in enumerate(phyloDistances) if loss_name == str(element).replace(".", "")+"distance"][0]

In [ ]:
distance = phyloDistances[distance_indx]
next_distance = phyloDistances[distance_indx+1] if distance_indx<len(phyloDistances)-1 else 0
abs_total_dist = 512
print('distance_indx', distance_indx, int(next_distance*abs_total_dist), int(distance*abs_total_dist))

In [ ]:
distance_indx

In [ ]:
phyloDistances[0]*abs_total_dist

In [31]:
import torch
import numpy as np

def deconv_orth_dist(kernel1, kernel2, stride = 2, padding = 1):
    assert (kernel1.shape[0] == kernel2.shape[0]) or (kernel1.shape[1] == kernel2.shape[1]) , "Kernels should be of compatible sizes" + str(kernel1.shape) + ", " + str(kernel2.shape)
    if kernel1.shape[1] != kernel2.shape[1]:
        kernel1 = kernel1.permute(1, 0, 2, 3)
        kernel2 = kernel2.permute(1, 0, 2, 3)
        print('permuted')
    [o_c, i_c, w, h] = kernel1.shape
    output = torch.conv2d(kernel1, kernel2, stride=stride, padding=padding)
    print(kernel1.shape)
#     print(target)
#     print(output)
    return torch.norm( output )

kernel = torch.rand((15,5,3,3)).cuda()
kernel2 = torch.rand((15,15,3,3)).cuda()
deconv_orth_dist(kernel,kernel2,1, 1)
print(kernel.shape)

permuted
torch.Size([5, 15, 3, 3])
torch.Size([15, 5, 3, 3])


In [17]:
print(target)

NameError: name 'target' is not defined